In [1]:
!pip install git+git://github.com/deepmind/jraph.git

  Cloning git://github.com/deepmind/jraph.git to /tmp/pip-req-build-3zyojk43
  Created wheel for jraph: filename=jraph-0.0.1.dev0-py3-none-any.whl size=70419 sha256=ea5c4b7fb908c2cf11a65702da3a3fcb55da757f61b287b74e95bc95ffe559b5
  Stored in directory: /tmp/pip-ephem-wheel-cache-sn6an42c/wheels/f2/97/65/e08bbe57dc5f8e486ef42a2e638f299550f065c65541e1bbc8
Successfully built jraph


In [ ]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install torch-geometric

Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html


In [ ]:
!pip install dm-haiku optax

In [ ]:
import jraph
import torch
import numpy as np
import networkx as nx
import community as community_louvain
from torch_geometric.data import InMemoryDataset, Data
from torch_geometric.utils import to_networkx
from networkx import karate_club_graph, to_numpy_matrix
from absl import app
import jax
import jax.numpy as jnp
import haiku as hk
from absl import app
import logging
from jax.interpreters.xla import DeviceArray
from jax.experimental import stax
from jax.experimental import optimizers
import torch
import numpy as np
import networkx as nx
import community as community_louvain
from torch_geometric.data import InMemoryDataset, Data
from torch_geometric.utils import to_networkx
import matplotlib.pyplot as plt
import logging
import tensorflow.compat.v1 as tf
from absl import app
import haiku as hk
import jax
import jax.numpy as jnp
import optax
import random
from decimal import Decimal
import timeit

In [ ]:
class KarateClub(InMemoryDataset):

    def __init__(self, transform=None):
        super(KarateClub, self).__init__('.', transform, None, None)

        G = nx.karate_club_graph()

        x = torch.eye(G.number_of_nodes(), dtype=torch.float)
        order = sorted(list(G.nodes()))
        adjency = to_numpy_matrix(G,nodelist=order)
        
        adj = nx.to_scipy_sparse_matrix(G).tocoo()
        row = torch.from_numpy(adj.row.astype(np.int64)).to(torch.long)
        col = torch.from_numpy(adj.col.astype(np.int64)).to(torch.long)
        edge_index = torch.stack([row, col], dim=0)

        
        y = [param['club'] for _, param in G.nodes().items()]
        class_map = {class_name: index for index, class_name in enumerate(np.unique(y))}
        y = list(map(lambda val: class_map[val], y))
        y = torch.from_numpy(np.array(y))
        # Select a single training node for each community
        # (we just use the first one).
        train_mask = torch.zeros(y.size(0), dtype=torch.bool)
        num_nodes = G.number_of_nodes()
        for i in range(int(y.max()) + 1):
            train_mask[(y == i).nonzero(as_tuple=False)[0]] = True

        data = Data(x=x, edge_index=edge_index, y=y, train_mask=train_mask,
                    adjency = adjency)

        self.data, self.slices = self.collate([data])

    def __repr__(self):
        return '{}()'.format(self.__class__.__name__)
dataset = KarateClub()
dataset = dataset[0]
dataset

In [ ]:
def get_zacharys_karate_club() -> jraph.GraphsTuple:
    
    social_graph = dataset.edge_index.t()
    n_club_members = dataset.num_nodes
    return jraph.GraphsTuple(
    n_node=jnp.asarray([n_club_members]),
    n_edge=jnp.asarray([len(social_graph)]),
      # One-hot encoding for nodes.
    nodes=jnp.eye(n_club_members),
      # No edge features.
    edges=None,
    globals=None,
    senders=jnp.asarray([edge[0] for edge in social_graph]),
    receivers=jnp.asarray([edge[1] for edge in social_graph]))

In [ ]:
def get_ground_truth_assignments_for_zacharys_karate_club() -> jnp.ndarray:    
    return jnp.array(np.array(dataset.y).reshape(-1, 1))

In [ ]:
def network_definition(graph: jraph.GraphsTuple) -> jraph.ArrayTree:
  def update_edge_fn(
      edge_features,
      sender_node_features,
      receiver_node_features,
      globals_):
    del sender_node_features
    del receiver_node_features
    del globals_
    return edge_features

  def update_node_fn(
      node_features,
      aggregated_sender_edge_features,
      aggregated_receiver_edge_features,
      globals_):
    del aggregated_sender_edge_features
    del aggregated_receiver_edge_features
    del globals_
    return hk.nets.MLP([34, 34, 2])(node_features)

  def update_globals_fn(
      aggregated_node_features,
      aggregated_edge_features,
      globals_):
    del aggregated_node_features
    del aggregated_edge_features
    return globals_

  gn = jraph.GraphNetwork(
    update_edge_fn=update_edge_fn,
    update_node_fn=update_node_fn,
    update_global_fn=update_globals_fn,
  )
  return gn(graph).nodes

In [ ]:
def print_graph(graph_touple, node_classes, title=None):
  graphs_nx = nx.DiGraph(list(zip(graph_touple.senders, graph_touple.receivers)))
  ax = plt.figure(figsize=(10, 10)).gca()
  nx.draw(graphs_nx, ax=ax, node_color=node_classes, cmap="Set3", arrows=None)
  if title:
    _ = ax.set_title(title)

In [ ]:
zacharys_karate_club = get_zacharys_karate_club()
labels = get_ground_truth_assignments_for_zacharys_karate_club()

In [ ]:
print_graph(zacharys_karate_club, np.array(labels.reshape(-1)), "Ground Truth")

In [ ]:
@jax.jit
def prediction_loss(params):
    logits = jax.nn.sigmoid(network.apply(params, zacharys_karate_club))
    return -jnp.mean(labels * jnp.log(logits) + (1 - labels) * jnp.log(1 - logits))

In [ ]:
  network = hk.without_apply_rng(hk.transform(network_definition))
  params = network.init(jax.random.PRNGKey(np.random.randint(0, 1000000, dtype="int64")), zacharys_karate_club)

  opt_init, opt_update = optax.adam(1e-2)
  opt_state = opt_init(params)

In [ ]:
@jax.jit
def update(params, opt_state):
    g = jax.grad(prediction_loss)(params)
    updates, opt_state = opt_update(g, opt_state)
    return optax.apply_updates(params, updates), opt_state

In [ ]:
@jax.jit
def accuracy(params):
    decoded_nodes = network.apply(params, zacharys_karate_club)
    return jnp.mean(jnp.int64(decoded_nodes > 0.5) == labels)

In [ ]:
#def train(network_definition, graph_touple, step_num=500, print_frequency=None):
  #network = hk.without_apply_rng(hk.transform(network_definition))
  #params = network.init(jax.random.PRNGKey(np.random.randint(0, 1000000, dtype="int64")), graph_touple)

  #opt_init, opt_update = optax.adam(1e-2)
  #opt_state = opt_init(params)

 #losses, accuracies = [], []
  #for i in range(step_num):
    #params, opt_state = update(params, opt_state)
    #step_loss = prediction_loss(params)
    #step_accuracy = accuracy(params)
    #losses.append(step_loss)
    #accuracies.append(step_accuracy)

    #if print_frequency and i % print_frequency == 0:
      #predicted = np.argmax(network.apply(params, graph_touple), axis=1)
      #print_graph(graph_touple, predicted, 'Step: %d, Loss: %.2E, Accuracy: %.2E' % (i, step_loss, step_accuracy))
  #return losses, accuracies

In [ ]:
import time
def train(network_definition,params, graph_touple, step_num=100, print_frequency=None):
  #network = hk.without_apply_rng(hk.transform(network_definition))
  

  opt_init, opt_update = optax.adam(1e-3)
  opt_state = opt_init(params)

  train_losses, train_accuracies,train_times = [], [], []
  predicted = None
  for i in range(step_num):
    start_time = time.time()
    params, opt_state = update(params, opt_state)
    train_times.append(time.time() - start_time)
    step_loss = prediction_loss(params)
    step_accuracy = accuracy(params)
    if np.isnan(step_loss):
      train_losses.append(train_losses[-1])
      train_accuracies.append(train_accuracies[-1])
    else:
      train_losses.append(step_loss)
      train_accuracies.append(step_accuracy)
      step_predicted = np.argmax(network.apply(params, graph_touple), axis=1)
      if not np.isnan(step_predicted).any():
        predicted = step_predicted

    if  i % 10 == 0 or i ==99:
      print_graph(graph_touple, predicted, 'Step: %d, Loss: %.2E, Accuracy: %.2E, Train Time: %.2E' % (i, train_losses[-1], train_accuracies[-1], train_times[-1]))
  return train_losses, train_accuracies, train_times

In [ ]:
params = network.init(jax.random.PRNGKey(np.random.randint(0, 1000000, dtype="int64")), zacharys_karate_club)
np.save('params_dump.npy', hk.data_structures.to_mutable_dict(params))
train_losses, train_accuracies, train_times, = train(network_definition,params, zacharys_karate_club, print_frequency=100)

In [ ]:
plt.plot(list(zip(train_losses, train_accuracies)))

In [ ]:
def predict(network_definition, graph_touple, params, print_result=False):
  start_time = time.time()
  predictionn_loss = prediction_loss(params)
  prediction_accuracy = accuracy(params)
  predicted = np.argmax(network.apply(params, graph_touple), axis=1)
  prediction_time = time.time() - start_time

  #if print_result:
    #print_graph(graph_touple, predicted, 'Predicted, Loss: %.2E, Accuracy: %.2E, Train Time: %.2E' % (prediction_loss, prediction_accuracy, prediction_time))
  return predictionn_loss, prediction_accuracy, prediction_time

In [ ]:
train_losses, train_accuracies, train_times = [], [], []
prediction_losses, prediction_accuracies, prediction_times = [], [], []

for i in range(25):
  params = network.init(jax.random.PRNGKey(np.random.randint(0, 1000000, dtype="int64")), zacharys_karate_club)
  %%timeit
  train_losses_values, train_accuracies_values, train_times_values = train(network_definition,params, zacharys_karate_club, print_frequency=100)
  train_losses.append(train_losses_values)
  train_accuracies.append(train_accuracies_values)
  train_times.append(train_times_values)

  prediction_losses_values, prediction_accuracies_values, prediction_times_values = predict(network_definition, zacharys_karate_club, params)
  prediction_losses.append(prediction_losses_values)
  prediction_accuracies.append(prediction_accuracies_values)
  prediction_times.append(prediction_times_values)

In [ ]:
train_losses = np.array(train_losses)
train_accuracies = np.array(train_accuracies)
train_times = np.array(train_times)
prediction_losses = np.array(prediction_losses)
prediction_accuracies = np.array(prediction_accuracies)
prediction_times = np.array(prediction_times)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
max_values = np.array(np.max(train_losses, axis=0), dtype=float)
min_values = np.array(np.min(train_losses, axis=0), dtype=float)
mean_values = np.array(np.mean(train_losses, axis=0), dtype=float)

x = np.arange(0, train_losses.shape[1])
ax.plot(x, max_values, color="tab:blue")
ax.plot(x, min_values, color="tab:blue")
ax.plot(x, mean_values, color="tab:blue")
ax.fill_between(x, min_values, max_values, color="tab:blue", alpha=0.3)
ax.set_title("Loss")
ax.set_xlabel("Step")
ax.set_ylabel("Loss value")
ax.grid()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
max_values = np.array(np.max(train_accuracies, axis=0), dtype=float)
min_values = np.array(np.min(train_accuracies, axis=0), dtype=float)
mean_values = np.array(np.mean(train_accuracies, axis=0), dtype=float)

x = np.arange(0, train_accuracies.shape[1])
ax.plot(x, max_values, color="tab:red")
ax.plot(x, min_values, color="tab:red")
ax.plot(x, mean_values, color="tab:red")
ax.fill_between(x, min_values, max_values, color="tab:red", alpha=0.3)
ax.set_title("Accuracy")
ax.set_xlabel("Step")
ax.set_ylabel("Accuracy value")
ax.grid()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
max_values = np.array(np.max(train_times, axis=0), dtype=float)
min_values = np.array(np.min(train_times, axis=0), dtype=float)
mean_values = np.array(np.mean(train_times, axis=0), dtype=float)

x = np.arange(0, train_times.shape[1])
ax.plot(x, max_values, color="tab:orange")
ax.plot(x, min_values, color="tab:orange")
ax.plot(x, mean_values, color="tab:orange")
ax.fill_between(x, min_values, max_values, color="tab:orange", alpha=0.3)
ax.set_title("Step time")
ax.set_xlabel("Step")
ax.set_ylabel("Step time")
ax.grid()

In [ ]:
print(f"Mean train time: {np.mean(train_times)}")

In [ ]:
print(f"Mean prediction loss: {np.mean(prediction_losses)}")
print(f"Mean prediction accuracy: {np.mean(prediction_accuracies)}")
print(f"Mean prediction time: {np.mean(prediction_times)}")